In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import pydicom
import numpy as np
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn as nn

KeyboardInterrupt: 

patient dicomdataset connects the dna to the images so we dont have to later

notice it has a structure of images,patient_id,gene_values, correponding genes

corresponding genes is purely informational, gene values are the y values per say, what we are trying to predict, paitent_id is the patient_id and images are a list of images we will use as multimodal input to discover/predict r values

In [8]:

from helper import PatientDicomDataset

transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
if not os.path.exists('data/Images/NSCLC Radiogenomics'):
         raise FileNotFoundError("The directory 'data/Images/NSCLC Radiogenomics' does not exist.")
if not os.path.exists('data/df_zscore.csv'):
         raise FileNotFoundError("The file 'data/df_zscore.csv' does not exist.")
# Verify that the dataset is properly loaded
dataset = PatientDicomDataset(root_dir='data/Images/NSCLC Radiogenomics', csv_path='data/df_zscore.csv', transform=transform)

# Check the length of the dataset to ensure it is not empty
if len(dataset) == 0:
    raise ValueError("The dataset is empty. Please check the root_dir and csv_path for correctness.")


images, patient_id, gene_values, gene_names = dataset[0]
print("Image tensor shape:", images[0].shape)

Loading CSV and images...
Image tensor shape: torch.Size([1, 288, 384, 384])


In [9]:
print("Patient ID:", patient_id)
print("Gene values:", gene_values)

Patient ID: R01-003
Gene values: gene
TOR1AIP1      -0.905057612885756
CHD7         -0.6035116464582104
ZFYVE26      -0.7228656453258673
NAA38        -0.4831588206955312
CASP2        -0.4219947274816687
                    ...         
MMP2         -0.2926089706377244
TPD52L1     0.009008742414673046
AMBRA1       -1.1338038819204568
GSTM3       -0.07446985032221468
SHROOM2     -0.42087371307454513
Name: R01-003, Length: 1000, dtype: object


split into training, validation and test sets

In [10]:
from torch.utils.data import random_split

# Define the split sizes
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Perform the split
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 91
Validation set size: 19
Test set size: 20


MedicalImageCNN	Per-image encoder → [B, 1, H, W] → [B, emb_dim]
CNNToRNA	Aggregates per-patient images and predicts RNA expression
train_loader	Should return [B, N, 1, H, W], [B, num_genes]

In [ ]:
#check if gpu is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
from model import MedicalImageCNN, CNNToRNA, train_model

from helper import collate_fn

# Create final model
model = CNNToRNA(
    cnn_encoder=MedicalImageCNN(output_dim=128),
    embedding_dim=128,
    output_dim=len(gene_names)
)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training params
num_epochs = 10
batch_size = 16  # Reduce batch size to fit within memory constraints

# DataLoaders (make sure you use collate_fn if images are lists!)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

train_losses, val_losses = train_model(
    model, train_loader, val_loader,
    criterion, optimizer, device,
    num_epochs=3
)

#construct the graph so we can visualize the training and validation loss
import matplotlib.pyplot as plt
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
# Save the model (for walking away lmao)
torch.save(model.state_dict(), 'model.pth')

RuntimeError: [enforce fail at alloc_cpu.cpp:116] data. DefaultCPUAllocator: not enough memory: you tried to allocate 294912 bytes.

test on test test set:

In [ ]:

from model import test_model

preds, labels = test_model(model, test_loader, device)

print("Test MSE:", mean_squared_error(labels, preds))
print("Test R² :", r2_score(labels, preds))  # Good for regression quality